In [ ]:
import loadAndClean
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn import cross_validation

In [ ]:
data = loadAndClean.loadAndClean()
data.describe()

In [ ]:
class baseline(object):
    def __init__(self):
        self.has_fit = False
        
    def fit(self, X_train, y_train, **kwargs):
        self.average_value = y_train.mean()

        self.has_fit = True

    def predict(self, X_test):
        if self.has_fit:
            return np.ones((len(X_test),)) * self.average_value
        return None
    
    def get_params(self, deep=False):
        return {}

alg = baseline()
scores = cross_validation.cross_val_score(alg, data, data['Average Medicare Payments Num'], scoring='mean_squared_error' ,cv=3)
print scores.mean()

In [ ]:
class grouped_baseline(object):
    def __init__(self):
        self.has_fit = False

    def fit(self, X_train, y_train, **kwargs):
        X_train = X_train.copy()
        X_train['Cost'] = y_train
        groups = X_train.groupby(['DRG Code'])

        # Average the cost for each DRG
        self.drg_costs = {}
        for ind,data in groups:
            self.drg_costs[ind] = data['Cost'].mean()

        self.has_fit = True

    def predict(self, X_test):
        if self.has_fit:
            return X_test['DRG Code'].apply(lambda x: self.drg_costs[x])
        return None

    def get_params(self, deep=False):
        return {}

alg = grouped_baseline()
#### This won't run because cross_val_score doesn't seem to be stratifying like it says it should be
scores = cross_validation.cross_val_score(alg, data, data['Average Medicare Payments Num'], scoring='mean_squared_error' ,cv=3)
print scores.mean()

In [ ]:
clf = RandomForestRegressor(n_estimators=500, n_jobs=4, verbose=3)
predictorsRan = ['Latitude','Longitude','DRG Code']
scores = cross_validation.cross_val_score(clf, data[predictorsRan], data["Average Medicare Payments Num"], scoring='mean_squared_error' ,cv=3)
print scores.mean()

baseline:   -57116130.0673
grouped_bl:
RFR:        -70821199.9133